# Обедающие философы (Threading version with timing)

In [35]:
import threading, random, time

class Filosoff(threading.Thread):
    running = True

    def __init__(self, xname, forkOne, forkTwo):
        super().__init__()
        self.name = xname
        self.forkOnLeft = forkOne
        self.forkOnRight = forkTwo
        self.f_eat_count = 0
        self.total_eating_time = 0
        self.last_eat_start = 0

    def run(self):
        while self.running:
            time.sleep(random.uniform(2,5))
            print(self.name, 'голодный.')
            self.dine()
        else:
            print(self.name, 'поел', self.f_eat_count, 'раз. Всего времени еды:', round(self.total_eating_time, 2), 'сек')
            return

    def dine(self):
        fork1, fork2 = self.forkOnLeft, self.forkOnRight

        while self.running:
            fork1.acquire(True)
            locked = fork2.acquire(False)
            if locked: break
            fork1.release()
            print(self.name, 'поменял порядок захвата.')
            fork1, fork2 = fork2, fork1
        else:
            return
            
        self.f_eat_count += 1
        print(self.name, 'начал есть.')
        self.last_eat_start = time.time()
        time.sleep(random.uniform(3,6))
        eating_duration = time.time() - self.last_eat_start
        self.total_eating_time += eating_duration
        print(self.name, 'наелся и думает. Ел', round(eating_duration, 2), 'сек')
        fork1.release()
        fork2.release()

In [36]:
def DiningFilosofs(numfilos):
    if (numfilos < 2) or (numfilos > 11):
       print('Число философов от 2 до 11!')
       return

    forks = [threading.Lock() for n in range(numfilos)]
    philosophNames = ('1:Кант:','2:Маркс:','3:Платон:','4:Руссо:','5:Сократ:',
                     '6:Пифагор:','7:Гегель:','8:Вольтер:','9:Декарт:',
                     '10:ЛаоЦзы:','11:Рассел:')
    
    philosophers = [Filosoff(philosophNames[i], forks[i%numfilos],
                   forks[(i+1)%numfilos]) for i in range(numfilos)]

    random.seed(1537)
    modelstart=time.perf_counter_ns()
    Filosoff.running = True
    for p in philosophers: p.start()
    time.sleep(40)
    Filosoff.running = False
    for p in philosophers: p.join()
    modelfini=time.perf_counter_ns()
    
    print("\n== Итоговые результаты ==")
    for p in philosophers:
        print(f"{p.name}: ел {p.f_eat_count} раз, общее время еды: {round(p.total_eating_time, 2)} сек")
    
    print("== Все закончили! Время =", (modelfini-modelstart)/1000000, "msec")

In [37]:
print("=== Тест с 3 философами ===")
DiningFilosofs(3)

print("\n=== Тест с 5 философами ===")
DiningFilosofs(5)

print("\n=== Тест с 10 философами ===")
DiningFilosofs(10)

=== Тест с 3 философами ===
3:Платон: голодный.
3:Платон: начал есть.
1:Кант: голодный.
2:Маркс: голодный.
2:Маркс: поменял порядок захвата.
3:Платон: наелся и думает. Ел 4.39 сек
1:Кант: начал есть.
2:Маркс: поменял порядок захвата.
3:Платон: голодный.
3:Платон: поменял порядок захвата.
1:Кант: наелся и думает. Ел 5.73 сек
2:Маркс: начал есть.
3:Платон: поменял порядок захвата.
1:Кант: голодный.
1:Кант: поменял порядок захвата.
2:Маркс: наелся и думает. Ел 4.7 сек
3:Платон: начал есть.
1:Кант: поменял порядок захвата.
2:Маркс: голодный.
2:Маркс: поменял порядок захвата.
3:Платон: наелся и думает. Ел 5.46 сек
1:Кант: начал есть.
2:Маркс: поменял порядок захвата.
1:Кант: наелся и думает. Ел 3.64 сек
2:Маркс: начал есть.
3:Платон: голодный.
1:Кант: голодный.
1:Кант: поменял порядок захвата.
2:Маркс: наелся и думает. Ел 4.42 сек
1:Кант: начал есть.
3:Платон: поменял порядок захвата.
2:Маркс: голодный.
1:Кант: наелся и думает. Ел 4.62 сек
3:Платон: начал есть.
2:Маркс: поменял порядок захв

# Обедающие философы (Asyncio version with timing)

In [38]:
import asyncio, random, time

class AsyncFilosoff:
    def __init__(self, name, lockA, lockB):
        self.name = name
        self.lockA = lockA
        self.lockB = lockB
        self.eat_count = 0
        self.total_eating_time = 0
        
    async def run(self, running):
        while running[0]:
            await asyncio.sleep(random.uniform(2,5))
            print(self.name, 'голодный.')
            await self.dine(running)
        print(self.name, 'поел', self.eat_count, 'раз. Всего времени еды:', round(self.total_eating_time, 2), 'сек')
    
    async def dine(self, running):
        async with self.lockA:
            async with self.lockB:
                if not running[0]:
                    return
                self.eat_count += 1
                print(self.name, 'начал есть.')
                eat_time = random.uniform(3,6)
                start_time = time.time()
                await asyncio.sleep(eat_time)
                self.total_eating_time += time.time() - start_time
                print(self.name, 'наелся и думает. Ел', round(eat_time, 2), 'сек')

In [39]:
async def run_async_philosophers(numfilos):
    if (numfilos < 2) or (numfilos > 11):
        print('Число философов от 2 до 11!')
        return
    
    forks = [asyncio.Lock() for _ in range(numfilos)]
    philosophNames = ('1:Кант:','2:Маркс:','3:Платон:','4:Руссо:','5:Сократ:',
                     '6:Пифагор:','7:Гегель:','8:Вольтер:','9:Декарт:',
                     '10:ЛаоЦзы:','11:Рассел:')
    
    philosophers = [AsyncFilosoff(philosophNames[i], forks[i%numfilos],
                   forks[(i+1)%numfilos]) for i in range(numfilos)]
    
    running = [True]
    tasks = [philo.run(running) for philo in philosophers]
    
    modelstart = time.perf_counter_ns()
    await asyncio.sleep(40)
    running[0] = False
    
    await asyncio.gather(*tasks)
    modelfini = time.perf_counter_ns()
    
    print("\n== Итоговые результаты ==")
    for philo in philosophers:
        print(f"{philo.name}: ел {philo.eat_count} раз, общее время еды: {round(philo.total_eating_time, 2)} сек")
    
    print("== Все закончили! Время =", (modelfini-modelstart)/1000000, "msec")

In [40]:
print("=== Тест с 3 философами ===")
await run_async_philosophers(3)

print("\n=== Тест с 5 философами ===")
await run_async_philosophers(5)

print("\n=== Тест с 10 философами ===")
await run_async_philosophers(10)

=== Тест с 3 философами ===
1:Кант: поел 0 раз. Всего времени еды: 0 сек
2:Маркс: поел 0 раз. Всего времени еды: 0 сек
3:Платон: поел 0 раз. Всего времени еды: 0 сек

== Итоговые результаты ==
1:Кант:: ел 0 раз, общее время еды: 0 сек
2:Маркс:: ел 0 раз, общее время еды: 0 сек
3:Платон:: ел 0 раз, общее время еды: 0 сек
== Все закончили! Время = 40043.197198 msec

=== Тест с 5 философами ===
1:Кант: поел 0 раз. Всего времени еды: 0 сек
2:Маркс: поел 0 раз. Всего времени еды: 0 сек
3:Платон: поел 0 раз. Всего времени еды: 0 сек
4:Руссо: поел 0 раз. Всего времени еды: 0 сек
5:Сократ: поел 0 раз. Всего времени еды: 0 сек

== Итоговые результаты ==
1:Кант:: ел 0 раз, общее время еды: 0 сек
2:Маркс:: ел 0 раз, общее время еды: 0 сек
3:Платон:: ел 0 раз, общее время еды: 0 сек
4:Руссо:: ел 0 раз, общее время еды: 0 сек
5:Сократ:: ел 0 раз, общее время еды: 0 сек
== Все закончили! Время = 40042.203053 msec

=== Тест с 10 философами ===
1:Кант: поел 0 раз. Всего времени еды: 0 сек
2:Маркс: пое